In [3]:
import numpy as np
import mnist
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [4]:
train_img = mnist.train_images()
train_lbl = mnist.train_labels()

test_img = mnist.test_images()
test_lbl = mnist.test_labels()

train_img = train_img.reshape((train_img.shape[0], train_img.shape[1] * train_img.shape[2]))

test_img = test_img.reshape((test_img.shape[0], test_img.shape[1] * test_img.shape[2]))

In [5]:
train_img = np.array(train_img, dtype='float32') / 255
train_lbl = np.array(train_lbl, dtype='int16')
test_img = np.array(test_img, dtype='float32') / 255
train_lbl = np.array(train_lbl, dtype='int16')

In [6]:
def compute_black(image):
    
    black = []
    
    for idx in range(image.shape[0]):
        
        b = np.empty((28,28))     
        x = image[idx].reshape((28,28)) 
        b[0, 0] = 1 if x[0, 0] == 0 else 0
        
        for i in range(1, 28):
            
            t = b[0, i - 1]
            b[0, i] = t + 1 if x[0, i] == 0 else t
            
        for i in range(1, 28):
            
            for j in range(28):
                
                if j == 0:
                    t = b[i - 1, j]
                    b[i, j] = t + 1 if x[i, j] == 0 else t
                
                else:
                    t = b[i - 1, j] + b[i, j - 1] - b[i - 1, j - 1]
                    b[i, j] = t + 1 if x[i, j] == 0 else t
                    
        black.append(b)
        
    return np.array(black, dtype='int')

In [7]:
train_img_black = compute_black(train_img)

In [12]:
k = 100
rect = []

while len(rect) != k:
    
    # random left point, length, and breadth
    topleft = np.random.randint(low=5, high=23, size=2)
    b = np.random.randint(low=5, high=28 - topleft[1])
    l = np.random.randint(low=5, high=28 - topleft[0])
    
    if b * l < 130 or b * l >170:
        continue
    
    # other corners
    topright = np.array([topleft[0] + l, topleft[1]])
    bottomleft = np.array([topleft[0], topleft[1] + b])
    bottomright = np.array([topleft[0] + l, topleft[1] + b])
    
    # mid points
    vtop = np.array([topleft[0] + l//2, topleft[1]])
    vbottom = np.array([topleft[0] + l//2, topleft[1]])
    
    hleft = np.array([topleft[0], topleft[1] + b//2])
    hright = np.array([topright[0], topright[1] + b//2])

    rect.append([topleft, topright, bottomleft, bottomright, vtop, vbottom, hleft, hright])

In [31]:
def HARR(X, b):
    
    _X = np.empty((X.shape[0], 200), dtype='int16')
    
    for i in range(X.shape[0]):
        
        # extract image
        img = b[i]
        
        # compute features
        for j in range(k):
            tl, tr, bl, br, vt, vb, hl, hr = rect[j]
            
            top = img[hr[0], hr[1]] - img[hl[0], hl[1]] - img[tr[0], tr[1]] + img[tl[0], tl[1]]
            bottom = img[br[0], br[1]] - img[bl[0], bl[1]] - img[hr[0], hr[1]] + img[hl[0], hl[1]]
            v_score = top - bottom
            
            left = img[vb[0], vb[1]] - img[bl[0], bl[1]] - img[vt[0], vt[1]] + img[tl[0], tl[1]]
            right = img[br[0], br[1]] - img[vb[0], vb[1]] - img[tr[0], tr[1]] + img[vt[0], vt[1]]
            h_score = left - right

            _X[i][j*2] = v_score
            _X[i][(j*2)+1] = h_score

    return _X

In [32]:
harr_train_img = HARR(train_img, train_img_black)

In [16]:
mnist_lr = LogisticRegression(n_jobs=-1)
mnist_lr.fit(harr_train_img, train_lbl)
yhat = mnist_lr.predict(harr_train_img)
sum(yhat == train_lbl)/len(train_lbl)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Train acc: 0.9090833333333334


In [33]:
test_img_black = compute_black(test_img)
test_img_black_harr = HARR(test_img, test_img_black)
yhat_test = mnist_lr.predict(test_img_black_harr)

In [29]:
(sum(yhat_test == test_lbl)/len(test_lbl))

0.9132